# Building Engineering Data Models with Helper Functions

In this notebook we will demonstrate an easy way to start building new distribution network models in the engineering format using new helper functions add in PowerModelsDistribution v0.9

In [1]:
using PowerModelsDistribution

┌ Info: Precompiling PowerModelsDistribution [d7431456-977f-11e9-2de3-97ff7677985e]
└ @ Base loading.jl:1260


First, we need a optimizer. In this case we will use Ipopt and initialize it with JuMP's `optimizer_with_attributes`, which we have exported from PowerModelsDistribution by default for the user

In [2]:
using LinearAlgebra: diagm
import Ipopt

ipopt_solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level"=>0, "tol"=>1e-6)

MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("print_level") => 0, MathOptInterface.RawParameter("tol") => 1.0e-6])

Initialize an empty `ENGINEERING` model

In [3]:
eng = Model()

Dict{String,Any} with 3 entries:
  "settings"   => Dict{String,Any}("sbase_default"=>1.0,"vbases_default"=>Dict{…
  "per_unit"   => false
  "data_model" => ENGINEERING

In this block we build a three bus network, with neutrals grounded at the source and loads.

We start with buses, with the sourcebus and loadbus having 4 terminals, with the last terminal grounded.

Then we add a generation source, in this case a voltage source, which is `WYE` configured by default, and therefore expects the last conductor to be a grounded neutral

We add two three phase lines to connect the buses `sourcebus`, `primary`, and `loadbus`. Note that none of the lines have a neutral conductor.

We finally add a three-phase load a the `loadbus` bus, but note again that this is a `WYE` configured load, and like the voltage source, this implies that the last conductor is a grounded neutral for the purposes of kron reduction (which is required by default until explicit 4-wire modeling is added to PowerModelsDistribution)
    
Lastly, we need to define the default vbase of the system at the `sourcebus`

In [4]:
add_bus!(eng, "sourcebus"; terminals=[1,2,3,4], grounded=[4])
add_bus!(eng, "primary"; terminals=[1,2,3])
add_bus!(eng, "loadbus"; terminals=[1,2,3,4], grounded=[4])

add_voltage_source!(eng, "source", "sourcebus", [1,2,3,4]; vm=[1, 1, 1])

add_linecode!(eng, "default", diagm(0=>fill(0.01, 3)), diagm(0=>fill(0.2, 3)))

add_line!(eng, "trunk", "sourcebus", "primary", [1,2,3], [1,2,3]; linecode="default")
add_line!(eng, "primary", "primary", "loadbus", [1,2,3], [1,2,3]; linecode="default")

add_load!(eng, "balanced", "loadbus", [1,2,3,4]; pd_nom=[5, 5, 5], qd_nom=[1, 1, 1])

add_vbase_default!(eng, "sourcebus", 1)

eng

Dict{String,Any} with 7 entries:
  "voltage_source" => Dict{String,Any}("source"=>Dict{String,Any}("source_id"=>"vo…
  "line"           => Dict{String,Any}("primary"=>Dict{String,Any}("xs"=>[0.2 0.2 …
  "settings"       => Dict{String,Any}("sbase_default"=>1.0,"vbases_default"=>D…
  "load"           => Dict{String,Any}("balanced"=>Dict{String,Any}("source_id"=>"…
  "bus"            => Dict{String,Any}("primary"=>Dict{String,Any}("source_id"=>"b…
  "per_unit"       => false
  "data_model"     => ENGINEERING

Running this case with OPF gives the results below

In [5]:
result = solve_mc_opf(eng, ACPPowerModel, ipopt_solver)

[warn | PowerModels]: Updated generator 1 cost function with order 3 to a function of order 2: [0.001, 0.0]

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



Dict{String,Any} with 8 entries:
  "solve_time"         => 3.70515
  "optimizer"          => "Ipopt"
  "termination_status" => LOCALLY_SOLVED
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 0.0150047
  "solution"           => Dict{String,Any}("voltage_source"=>Dict{String,Any}("sou…
  "objective_lb"       => -Inf

In the following example, we provide examples of a wider range of component types that can be added using helper functions to give a flavor of what is possible in PowerModelsDistribution v0.9

In [6]:
eng2 = deepcopy(eng)

add_bus!(eng2, "ttbus"; terminals=[1,2,3,4], grounded=[4])

add_transformer!(eng2, "tx1", "sourcebus", "ttbus", [1,2,3,4], [1,2,3,4])

add_bus!(eng2, "loadbus2"; terminals=[1,2,3,4], grounded=[4])

add_switch!(eng2, "breaker", "ttbus", "loadbus2", [1,2,3], [1,2,3])

add_load!(eng2, "tload", "loadbus2", [1,2,3,4]; pd_nom=[5, 5, 5], qd_nom=[1, 1, 1])

add_generator!(eng2, "secondary", "loadbus2", [1,2,3,4]; cost_pg_parameters=[0.0, 1.2, 0])

add_shunt!(eng2, "cap", "loadbus2", [1,2,3,4]; bs=diagm(0=>fill(1, 3)))

eng2

Dict{String,Any} with 11 entries:
  "bus"            => Dict{String,Any}("primary"=>Dict{String,Any}("source_id"=>"b…
  "settings"       => Dict{String,Any}("sbase_default"=>1.0,"vbases_default"=>D…
  "switch"         => Dict{String,Any}("breaker"=>Dict{String,Any}("source_id"=>"s…
  "generator"      => Dict{String,Any}("secondary"=>Dict{String,Any}("source_id"=>…
  "voltage_source" => Dict{String,Any}("source"=>Dict{String,Any}("source_id"=>"vo…
  "line"           => Dict{String,Any}("primary"=>Dict{String,Any}("xs"=>[0.2 0.2 …
  "per_unit"       => false
  "data_model"     => ENGINEERING
  "transformer"    => Dict{String,Any}("tx1"=>Dict{String,Any}("source_id"=>"trans…
  "shunt"          => Dict{String,Any}("cap"=>Dict{String,Any}("source_id"=>"shunt…
  "load"           => Dict{String,Any}("tload"=>Dict{String,Any}("source_id"=>"loa…

In [7]:
result2 = solve_mc_opf(eng2, ACPPowerModel, ipopt_solver)

[warn | PowerModels]: active generators found at bus 4, updating to bus type from 1 to 2
[warn | PowerModels]: Updated generator 1 cost function with order 3 to a function of order 2: [0.0012, 0.0]
[warn | PowerModels]: Updated generator 2 cost function with order 3 to a function of order 2: [0.001, 0.0]


Dict{String,Any} with 8 entries:
  "solve_time"         => 0.087898
  "optimizer"          => "Ipopt"
  "termination_status" => LOCALLY_SOLVED
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => -83.3003
  "solution"           => Dict{String,Any}("voltage_source"=>Dict{String,Any}("sou…
  "objective_lb"       => -Inf